<a href="https://colab.research.google.com/github/ashishar/data_science_codes/blob/main/King_county_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
#import kaggle dataset
! pip install -q kaggle
from google.colab import files
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download harlfoxem/housesalesprediction



mkdir: cannot create directory ‘/root/.kaggle’: File exists
  0% 0.00/780k [00:00<?, ?B/s]
100% 780k/780k [00:00<00:00, 120MB/s]


In [40]:
! unzip housesalesprediction.zip
!ls

Archive:  housesalesprediction.zip
replace kc_house_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: kc_house_data.csv       
housesalesprediction.zip  kaggle.json  kc_house_data.csv  sample_data


In [41]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [44]:
nRowsRead = 1000 # specify 'None' if want to read whole file
# kc_house_data.csv has 21613 rows in reality, but we are only loading/previewing the first 1000 rows
data_houseing = pd.read_csv('kc_house_data.csv', delimiter=',', nrows = nRowsRead)
data_houseing.dataframeName = 'kc_house_data.csv'
nRow, nCol = data_houseing.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 1000 rows and 21 columns


In [45]:
data_houseing.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [46]:
plotPerColumnDistribution(data_houseing, 10, 5)

NameError: ignored